In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Prepare the initial dataframe
names = ['Tommy','A','B','C','D','E','F','G','H', 'I','J','K','L','M','N','O','P','Q','R','S','T']

################################# Parameters #################################
days = 30
p = 0.02

Next_State_ModelSIR = dict({"I1":"I2","I2":"I3","I3":"R","R":"R"})
Next_State_ModelSIRS = dict({"I1":"I2","I2":"I3","I3":"R1","R1":"R2","R2":"R3","R3":"S"})
Next_State_ModelSIS = dict({"I1":"I2","I2":"I3","I3":"S"})
Next_State_ModelSEIR = dict({"E1":"E2", "E2":"I1","I1":"I2","I2":"I3","I3":"R", "R":"R"})

NumSim = 30000  
AnswerQs2 = 0
AnswerQs3 = 0
AnswerQs1 = pd.DataFrame({'Probability':np.zeros(21,dtype='float')})
Next_State_Model = Next_State_ModelSIR


In [ ]:
################################################################## Answer for Qs 1 to 3 ##################################################################

for n in range(NumSim):
    if(n%1000) == 0 : print(n)
    
    kids = pd.DataFrame({'names':names})
    for i in range(days):
        kids["day" + str(i+1)] = "S"

    kids.loc[kids["names"]=="Tommy","day1"] = "I1"

    for i in range(1,days):    
        
        #Update status for Infected kids
        kids["day"+ str(i+1)] = np.where(kids["day"+ str(i)]=="S","S",kids["day"+ str(i)]\
                                         .map(Next_State_Model))
        
        #Check if there are kids who can infect or there any susceptible population
        if (kids.loc[kids["day" + str(i)].isin(["I1","I2","I3"])].shape[0] > 0) & \
        (kids.loc[kids["day" + str(i)]=="S"].shape[0] > 0):
            # Generate a random matrix of size #Susceptible by #Infected
            random_mat = np.random.rand(kids.loc[kids["day" + str(i)]=="S"].shape[0],\
                                        kids.loc[kids["day" + str(i)].isin(["I1","I2","I3"])].shape[0])
            #Take row-wise minimum to see if any susceptible got infected
            random_mat = np.min(random_mat, axis = 1)
            NumSusceptible2Infected = random_mat[random_mat<=p].size

            if NumSusceptible2Infected > 0:
                kids.sort_values("day" + str(i),ascending=False, inplace=True)
                
                for j in range(NumSusceptible2Infected):
                    if kids.iloc[j,i]=="S":
                        kids.iloc[j,i+1] ="I1"
    
    AnswerQs1.iloc[kids.loc[kids["day2"] != "S"].shape[0]-1,0] += 1
    AnswerQs2 += kids.loc[kids['day2'] == "I1"].shape[0]
    AnswerQs3 += kids.loc[kids['day3'] == "I1"].shape[0]

AnswerQs1 = AnswerQs1/NumSim
AnswerQs2 = AnswerQs2/NumSim
AnswerQs3 = AnswerQs3/NumSim



In [ ]:
################################################################## Answer for Qs 4 ##################################################################
#Add code for plots
def plot_graphs(S,E,I,R,Days,model_name):
    
    t = np.linspace(0, Days,Days)
    
    fig = plt.figure(facecolor='w')
    ax = fig.add_subplot(111, facecolor='#dddddd', axisbelow=True)
    
    ax.plot(t, S, 'b', alpha=0.5, lw=2, label='Susceptible')
    ax.plot(t, I, 'r', alpha=0.5, lw=2, label='Infected')
    if E.shape[0] == Days:
        ax.plot(t, E, 'y', alpha=0.5, lw=2, label='Incubating')
    if R.shape[0] == Days:
        ax.plot(t, R, 'g', alpha=0.5, lw=2, label='Recovered')

    ax.set_xlabel('Time /days')
    ax.set_ylabel('Number Infected')
    ax.set_ylim(0,25)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)
    
    plt.title("{} Model".format(model_name))
    
    for spine in ('top', 'right', 'bottom', 'left'):
        ax.spines[spine].set_visible(False)
    plt.show()
    
def RunPandemicSim(days,NumSim,Next_State_Model,names,p_infect,TransitionFromS):
    col = []
    
    for i in range(days):
        col.append("day" + str(i+1))
    
    SimResults_S = pd.DataFrame(columns=col)
    SimResults_E = pd.DataFrame(columns=col)
    SimResults_I = pd.DataFrame(columns=col)
    SimResults_R = pd.DataFrame(columns=col)
    
    for n in range(NumSim):
        if(n%1000) == 0 : print(n)
        
        kids = pd.DataFrame({'names':names})
        for i in range(days):
            kids["day" + str(i+1)] = "S"            
        kids.loc[kids["names"]=="Tommy","day1"] = "I1" 
        
        for i in range(1,days):    
            
            #Update status for Infected kids
            kids["day"+ str(i+1)] = np.where(kids["day"+ str(i)]=="S","S",kids["day"+ str(i)] \
                                             .map(Next_State_Model))
            
            #Check if there are kids who can infect or there any susceptible population
            if (kids.loc[kids["day" + str(i)].isin(["I1","I2","I3"])].shape[0] > 0) \
            & (kids.loc[kids["day" + str(i)]=="S"].shape[0] > 0):
                
                # Generate a random matrix of size #Susceptible by #Infected
                random_mat = np.random.rand(kids.loc[kids["day" + str(i)]=="S"].shape[0],\
                                            kids.loc[kids["day" + str(i)].isin(["I1","I2","I3"])].shape[0])
                
                #Take row-wise minimum to see if any susceptible got infected
                random_mat = np.min(random_mat, axis = 1)
                NumSusceptible2Infected = random_mat[random_mat<=p_infect].size
    
                if NumSusceptible2Infected > 0:
                    kids.sort_values("day" + str(i),ascending=False, inplace=True)
                    
                    for j in range(NumSusceptible2Infected):
                        if kids.iloc[j,i] == "S":
                            kids.iloc[j,i+1] = TransitionFromS
                            
        data_to_append_S = {}
        data_to_append_E = {}
        data_to_append_I = {}
        data_to_append_R = {}

        arr_S = np.where(kids.iloc[:,1:] == "S",1,0).sum(axis = 0)
        arr_E = np.where(kids.iloc[:,1:] == "E1",1,0).sum(axis = 0) + \
                np.where(kids.iloc[:,1:] == "E2",1,0).sum(axis = 0)
        arr_I = np.where(kids.iloc[:,1:] == "I1",1,0).sum(axis = 0) + \
                np.where(kids.iloc[:,1:] == "I2",1,0).sum(axis = 0) + \
                np.where(kids.iloc[:,1:] == "I3",1,0).sum(axis = 0)
        arr_R = np.where(kids.iloc[:,1:] == "R",1,0).sum(axis = 0) + \
                np.where(kids.iloc[:,1:] == "R1",1,0).sum(axis = 0) + \
                np.where(kids.iloc[:,1:] == "R2",1,0).sum(axis = 0) + \
                np.where(kids.iloc[:,1:] == "R3",1,0).sum(axis = 0)
        
        for i in range(len(SimResults_S.columns)):
            data_to_append_S[SimResults_S.columns[i]] = arr_S[i]
            data_to_append_E[SimResults_E.columns[i]] = arr_E[i]
            data_to_append_I[SimResults_I.columns[i]] = arr_I[i]
            data_to_append_R[SimResults_R.columns[i]] = arr_R[i]
        
        SimResults_S = SimResults_S.append(data_to_append_S, ignore_index = True)   
        SimResults_E = SimResults_E.append(data_to_append_E, ignore_index = True)   
        SimResults_I = SimResults_I.append(data_to_append_I, ignore_index = True)   
        SimResults_R = SimResults_R.append(data_to_append_R, ignore_index = True)   
          
    return SimResults_S, SimResults_E, SimResults_I, SimResults_R



In [ ]:
SimResults_SIR_S,_,SimResults_SIR_I,SimResults_SIR_R = RunPandemicSim(30,30000,Next_State_ModelSIR,names,p,"I1")

plot_graphs(SimResults_SIR_S.mean(axis = 0),\
            pd.DataFrame(),SimResults_SIR_I.mean(axis = 0),\
            SimResults_SIR_R.mean(axis = 0),30,"SIR")

SimResults_SIS_S,_,SimResults_SIS_I,_ = RunPandemicSim(30,30000,Next_State_ModelSIS,names,p,"I1")

plot_graphs(SimResults_SIS_S.mean(axis = 0),\
            pd.DataFrame(),SimResults_SIS_I.mean(axis = 0),\
            pd.DataFrame(),30,"SIS")

SimResults_SIRS_S,_,SimResults_SIRS_I,SimResults_SIRS_R = RunPandemicSim(30,30000,Next_State_ModelSIRS,names,p,"I1")
plot_graphs(SimResults_SIRS_S.mean(axis = 0),\
            pd.DataFrame(),SimResults_SIRS_I.mean(axis = 0),\
            SimResults_SIRS_R.mean(axis = 0),30,"SIRS")

SimResults_SEIR_S, SimResults_SEIR_E, SimResults_SEIR_I, SimResults_SEIR_R = RunPandemicSim(30,30000,Next_State_ModelSEIR,names,p,"E1")
plot_graphs(SimResults_SEIR_S.mean(axis = 0), \
            SimResults_SEIR_E.mean(axis = 0), \
            SimResults_SEIR_I.mean(axis = 0), SimResults_SEIR_R.mean(axis = 0),30,"SEIR")


SimResults_SIR_S,_,SimResults_SIR_I,SimResults_SIR_R = RunPandemicSim(30,30000,Next_State_ModelSIR,names,p*2,"I1")
plot_graphs(SimResults_SIR_S.mean(axis = 0),\
            pd.DataFrame(),SimResults_SIR_I.mean(axis = 0),\
            SimResults_SIR_R.mean(axis = 0),30,"SIR")

SimResults_SIS_S,_,SimResults_SIS_I,_ = RunPandemicSim(30,30000,Next_State_ModelSIS,names,p*2,"I1")
plot_graphs(SimResults_SIS_S.mean(axis = 0),\
            pd.DataFrame(),SimResults_SIS_I.mean(axis = 0),\
            pd.DataFrame(),30,"SIS")

SimResults_SIRS_S,_,SimResults_SIRS_I,SimResults_SIRS_R = RunPandemicSim(30,30000,Next_State_ModelSIRS,names,p*2,"I1")
plot_graphs(SimResults_SIRS_S.mean(axis = 0),\
            pd.DataFrame(),SimResults_SIRS_I.mean(axis = 0),\
            SimResults_SIRS_R.mean(axis = 0),30,"SIRS")

SimResults_SEIR_S, SimResults_SEIR_E, SimResults_SEIR_I, SimResults_SEIR_R = RunPandemicSim(30,30000,Next_State_ModelSEIR,names,p*2,"E1")
plot_graphs(SimResults_SEIR_S.mean(axis = 0), \
            SimResults_SEIR_E.mean(axis = 0), \
            SimResults_SEIR_I.mean(axis = 0), SimResults_SEIR_R.mean(axis = 0),30,"SEIR")








